In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.utils import to_categorical 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Đọc dữ liệu
df = pd.read_csv('Reviews.csv')

# Chuyển đổi thời gian thành datetime
df['Time'] = pd.to_datetime(df['Time'], unit='s')

# Lọc các bản ghi của năm 2012
df_2012 = df[df['Time'].dt.year == 2012]

# Gán nhãn cảm xúc
def score_to_label(score):
    if score >= 4:
        return 'positive'
    elif score == 3:
        return 'neutral'
    else:
        return 'negative'

df_2012.loc[:, 'Label'] = df_2012['Score'].apply(score_to_label)

# Giữ lại các cột cần thiết
df_2012 = df_2012[['ProductId', 'Text', 'Label']]

# ===== TIỀN XỬ LÝ VĂN BẢN =====
# Khởi tạo các công cụ tiền xử lý
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Hàm tiền xử lý
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = " ".join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])
    return text

df_2012['Text'] = df_2012['Text'].apply(clean_text)

# ===== CHUẨN BỊ DỮ LIỆU =====
X = df_2012['Text']
y = df_2012['Label']

# Mã hóa nhãn thành số và one-hot
y = y.map({'positive': 0, 'neutral': 1, 'negative': 2})
y = to_categorical(y, num_classes=3)

# Tokenizer văn bản
max_len = 200
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)

X_seq = tokenizer.texts_to_sequences(X)
X_pad = pad_sequences(X_seq, maxlen=max_len)

# Chia tập train/test
X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.2, random_state=42)

# ===== OVERSAMPLING =====
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

# Chuyển y_train từ one-hot về label
y_train_labels = np.argmax(y_train, axis=1)

# Áp dụng oversampling
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled_labels = ros.fit_resample(X_train, y_train_labels)

# Chuyển về one-hot lại
y_train_resampled = to_categorical(y_train_resampled_labels, num_classes=3)

# Thống kê sau oversampling
print("Phân phối lớp sau oversampling:", Counter(np.argmax(y_train_resampled, axis=1)))

# ===== XÂY DỰNG MÔ HÌNH LSTM =====
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_len))
model.add(LSTM(256, dropout=0.3, recurrent_dropout=0.3))
model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax'))

# Biên dịch mô hình
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Tóm tắt mô hình
model.summary()

# Thiết lập EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# ===== HUẤN LUYỆN MÔ HÌNH =====
history = model.fit(
    X_train_resampled, y_train_resampled,
    epochs=3,
    batch_size=64,
    validation_data=(X_test, y_test),
    callbacks=[early_stop],
)

# ===== ĐÁNH GIÁ MÔ HÌNH =====
score, accuracy = model.evaluate(X_test, y_test, batch_size=64)
print(f'Test accuracy: {accuracy * 100:.2f}%')

# Tính precision, recall, f1-score
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

print("\n=== Báo cáo Precision / Recall / F1-score ===")
print(classification_report(y_true, y_pred, target_names=['positive', 'neutral', 'negative']))

# Thống kê số lượng đánh giá
product_summary = df_2012['ProductId'].value_counts()

print("\nTóm tắt số lượng đánh giá và cảm xúc của 10 sản phẩm đã xử lý:")
for product_id, count in product_summary.items():
    print(f"Product ID: {product_id} — Số lượng đánh giá: {count}")
    label_counts = df_2012[df_2012['ProductId'] == product_id]['Label'].value_counts()
    for label, label_count in label_counts.items():
        print(f"    {label}: {label_count}")


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 embedding (Embedding)       (None, 200, 128)          15220224  
                                                                 
 lstm (LSTM)                 (None, 256)               394240    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 3)                 771       
                                                                 
=================================================================
Total params: 15,615,235
Trainable params: 15,615,235
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
5683/5683 [==============================] - 7064s 1s/step - loss: 0.3904 - accuracy: 0.8524 - val_loss: 0.4609 - val_accuracy: 0.8674
Epoch 2/3
5683/5683 [==============================] - 7075s 1s/step - loss: 0.1295 - accuracy: 0.9564 - val_loss: 0.4920 - val_accuracy: 0.8824
Epoch 3/3
5683/5683 [==============================] - 7159s 1s/step - loss: 0.0620 - accuracy: 0.9799 - val_loss: 0.6333 - val_accuracy: 0.8836
621/621 [==============================] - 108s 173ms/step - loss: 0.4609 - accuracy: 0.8674
Test accuracy: 86.74%
1242/1242 [==============================] - 164s 131ms/step

=== Báo cáo Precision / Recall / F1-score ===
              precision    recall  f1-score   support

    positive       0.95      0.91      0.93     30260
     neutral       0.47      0.59      0.52      3131
    negative       0.74      0.78      0.76      6341

    accuracy                           0.87     39732
   macro avg       0.72      0.76      0.74     39732
weighted avg       0.88      0.87      0.87     39732